In [1]:
import numpy as np

from Animation import AnimateSolution2D, AnimateSolution3D
from CrankNicholsonSolver import Solver

# Условие задачи
![title](Task.png)

# Решение уравнения теплопроводности с источником тепла в 2D области
## Интенсивность источника тепла
$$ƒ(x, y, t) = \left( 1 - \frac{x^{2}}{L^{2}} \right)*\left( 1 - \frac{y^{2}}{L^{2}} \right)$$

In [2]:
def f(x, y):
    return (1 - x ** 2 / L ** 2) * (1 - y ** 2 / L ** 2)

## Параметры задачи
 $L = 1; \ T = 1; \ dt = 0.01 $

In [3]:
L = 1

dt = 0.01
T = 1

nx = 100
ny = 100
x0, x1, y0, y1 = (-L, L, -L, L)

## Решение уравнения локально-одномерным методом
На каждом шаге по времени отдельно рассматриваем распространение тепла по $x$ и по $y$.
В качестве схемы используем схему Кранка-Николсона

In [4]:
%%time
x_solvers = [Solver(x0, x1, nx, dt) for _ in range(ny)]
y_solvers = [Solver(y0, y1, ny, dt) for _ in range(nx)]

X, Y = np.meshgrid(x_solvers[0].xs, y_solvers[0].xs)
frames = []
solution = np.vectorize(f, otypes=[float])(X, Y)

for i in range(int(T / dt)):
    frames.append(np.copy(solution))
    for j, s in enumerate(x_solvers):
        solution[j + 1, :] = s.do_step(solution[j + 1, :])
    for j, s in enumerate(y_solvers):
        solution[:, j + 1] = s.do_step(solution[:, j + 1])

CPU times: user 1.24 s, sys: 620 ms, total: 1.86 s
Wall time: 572 ms


## Отображение полученного решения
### Распространение тепла в 3D


In [5]:
AnimateSolution3D(X, Y, frames)

### Распространение тепла в 2D

In [6]:
AnimateSolution2D(X, Y, frames)

invalid command name "4716024512_on_timer"
    while executing
"4716024512_on_timer"
    ("after" script)


In [8]:
import matplotlib.pyplot as plt

temperature_in_center = np.zeros(len(frames))

for i in range(len(frames)):
    temperature_in_center[i] = np.array(frames[i])[int(nx / 2), int(ny / 2)]

t = np.arange(0, T, dt)
plt.plot(t, temperature_in_center)
plt.plot(t, np.exp(-4.5 * t), ':')
plt.title('Temperature in the center')
plt.xlabel('T')
plt.ylabel('time')
plt.show()